In [1]:
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup
import json
from urllib.request import urlopen
from tqdm import tqdm_notebook

In [2]:
df_debut = pd.read_csv("../data/debut_album_1118.csv")
df_debut.head()

,artist,album,genre,single_count
0,2Cellos,2Cellos,"Cello rock, classical crossover",3
1,Deep Dark Robot,8 Songs About a Girl,"Post-grunge, punk blues",0
2,The Young Professionals,"9am to 5pm, 5pm to Whenever\r(09:00 to 17:00, ...",Electronic,0
3,Flash Bang Grenada,10 Haters,Alternative hip hop,0
4,Her Majesty & the Wolves,111,"Dance, pop",3


In [5]:
def get_xxl_articles():
    
    df_articles = pd.DataFrame(columns = ['artist', 'album', 'xxl_frequency'])
    
    for i in tqdm_notebook(range(len(list(df_debut['artist'])))):
        
        artist = df_debut['artist'][i]
        album = df_debut['album'][i]

        artist_ = artist.replace(" ", "%20")
        album_ = album.replace(" ", "%20")

        keyword = artist_ + "%20" + album_

        url = 'http://www.xxlmag.com/rest/carbon/api/searchfor/widgetsearch/?s=' + keyword + '&searchpage=1'
        response = requests.get(url).json()

        articles = response['data']['carbonwidget/searchnative-1']['params']['total']

        data = {
                'artist': artist,
                'album' : album,
                'xxl_frequency' : articles
            }

#         print(data)

        df_articles.loc[len(df_articles)] = data
    
    return df_articles

In [6]:
xxl_articles = get_xxl_articles()

HBox(children=(IntProgress(value=0, max=1949), HTML(value='')))

In [7]:
print(xxl_articles.shape)
xxl_articles.head()

(1949, 3)


,artist,album,xxl_frequency
0,2Cellos,2Cellos,0
1,Deep Dark Robot,8 Songs About a Girl,14
2,The Young Professionals,"9am to 5pm, 5pm to Whenever\r(09:00 to 17:00, ...",0
3,Flash Bang Grenada,10 Haters,0
4,Her Majesty & the Wolves,111,5408


In [17]:
xxl_articles.loc[528, 'artist'] = 'Eve Ai'
xxl_articles.loc[528, 'album'] = 'If You Love Me'
xxl_articles.loc[1142, 'artist'] = 'Milica Pavlovia'
xxl_articles.loc[1142, 'album'] = 'Gover Tela Body Language'
xxl_articles.loc[1409, 'artist'] = 'Various Artists'
xxl_articles.loc[1409, 'album'] = 'Fresh Filter : Volumn 1'

In [18]:
xxl_articles.to_csv("../data/buzz_xxl.csv", index=False, encoding='UTF-8')

In [19]:
df = pd.read_csv("../data/buzz_xxl.csv", encoding="UTF-8")

In [20]:
print(df.shape)
df.head()

(1949, 3)


,artist,album,xxl_frequency
0,2Cellos,2Cellos,0
1,Deep Dark Robot,8 Songs About a Girl,14
2,The Young Professionals,"9am to 5pm, 5pm to Whenever\r(09:00 to 17:00, ...",0
3,Flash Bang Grenada,10 Haters,0
4,Her Majesty & the Wolves,111,5408


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1949 entries, 0 to 1948
Data columns (total 3 columns):
artist           1949 non-null object
album            1949 non-null object
xxl_frequency    1949 non-null int64
dtypes: int64(1), object(2)
memory usage: 45.8+ KB


### Upload to MySQL database

In [22]:
import sqlalchemy, pickle
from sqlalchemy import create_engine

In [23]:
pw = pickle.load(open("mysql_pw.pickle", "rb"))

In [24]:
engine = sqlalchemy.create_engine("mysql+mysqldb://root:" + pw + "@52.78.44.120/project_rookie")

In [25]:
df.to_sql(name="buzz_xxl", con=engine, if_exists='replace')